<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

I have chosen to create a class which will include the necessary functions to geocode and process the collected data.

First, let's import the necessary libraries.

In [1]:
import json
import xml
import requests
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import geocoder

<h2>PART 1</h2>
In this part I am creating a class that will do the actual work.

Functions are the following:
<ul>
 	<li><strong>scrape</strong> for web scraping data from Wikipedia</li>
 	<li><strong>savetoCSV </strong>to save the data to a file</li>
 	<li><strong>geocodetheCoords</strong> to convert the postal data to latitude and longitude.</li>
    <li><strong>ifGeocoderFails</strong> to import latitudes and longitudes from a provided file. This is needed in case Geocoder fails (in my case, it sometimes failed to deliver). Then I use pd.merge to relate the coordinates with the scraped dataset.</li>
</ul>

In [2]:
class Scraper:
    postcode, borough, neighborhood = [], [], []
    
    def __init(self, df):
        self.df = df
        
    def scrape(self, url):
        soup = BeautifulSoup(url,'xml')
        table = soup.find('table')

        self.postcode, self.borough, self.neighborhood = [], [], []

        for tr_cell in table.find_all('tr'):
            
            count = 1
            PostcodeInd, BoroughInd, NeighborhoodInd = -1, -1, -1
            
            for td_cell in tr_cell.find_all('td'):
                if count == 1: 
                    PostcodeInd= td_cell.text
                if count == 2: 
                    BoroughInd= td_cell.text
                    tag_a_Borough = td_cell.find('a')
                    
                if count == 3: 
                    NeighborhoodInd = str(td_cell.text).strip()
                    tag_a_Neighborhood = td_cell.find('a')
                    
                count +=1
                
            if (PostcodeInd== 'Not assigned' or BoroughInd== 'Not assigned' or NeighborhoodInd == 'Not assigned'): 
                continue
            try:
                if ((tag_a_Borough is None) or (tag_a_Neighborhood is None)):
                    continue
            except:
                pass
            if(PostcodeInd== -1 or BoroughInd == -1 or NeighborhoodInd == -1):
                continue
                
            self.postcode.append(PostcodeInd)
            self.borough.append(BoroughInd)
            self.neighborhood.append(NeighborhoodInd)
            
        uPostal = set(self.postcode)
        print('Number of unique postal codes:', len(uPostal))
        uBorough, uNeighborhood, uPostcode = [], [], []
        
        for pcElement in uPostal:
            p = ''; b = ''; n = ''; 
            for postcode_idx, postcode_element in enumerate(self.postcode):
                if pcElement  == postcode_element:
                    p = postcode_element;
                    b = self.borough[postcode_idx]
                    if n == '': 
                        n = self.neighborhood[postcode_idx]
                    else:
                        n = n + ', ' + self.neighborhood[postcode_idx]
            uPostcode.append(p)
            uBorough.append(b)
            uNeighborhood.append(n)
            
        temp = {'Postcode': uPostcode, 'Borough' :uBorough, 'Neighborhood' :uNeighborhood}
        self.df = pd.DataFrame.from_dict(temp)
        
        return self.df, uPostcode
    
    def savetoCSV(self):
        self.df.to_csv('torontoData.csv')

    def geocodetheCoords(self):
        
        latitude = []
        longitude = []
        for elem in t[1]:
            lat_lng_coords = None

            while (lat_lng_coords is None):
                g = geocoder.google('{}, Toronto, Ontario'.format(elem))
                lat_lng_coords = g.latlng
                # print(lat_lng_coords)
        
            latitude.append(lat_lng_coords[0])
            longitude.append(lat_lng_coords[1])
            print(elem, 'is saved.')
        return latitude, longitude
    
    def ifGeocoderFails(self):
        importFromFile = pd.read_csv("https://cocl.us/Geospatial_data")
        return importFromFile

<h2>PART 2</h2>
In this part I execute the above-prescribed actions.

In [3]:
Toronto = Scraper()
t = Toronto.scrape(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text)


Toronto.savetoCSV()
print(t[0].head())

Number of unique postal codes: 84
           Borough                                       Neighborhood Postcode
0       North York                                         Henry Farm      M2J
1  Central Toronto                                      Lawrence Park      M4N
2        Etobicoke                                      Markland Wood      M9C
3        Etobicoke                                   Islington Avenue      M9A
4      Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...      M1P


In [4]:
#coords = Toronto.geocodetheCoords()
#t[0]['Latitude'] = coords[0]
#t[0]['Longitude'] = coords[1]

if geocoder fails, let's import the coordinates from the file.

In [5]:
coords = Toronto.ifGeocoderFails()
print(coords.head())

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [6]:
#Let's sort the data 
coords.sort_values(by=['Postal Code'])
coords.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
t[0].sort_values(by=['Postcode'])

#t[0]['Latitude'] = coords['Latitude']
#t[0]['Longitude'] = coords['Longitude']

expected_result = pd.merge(t[0], coords, on = 'Postcode', how = 'left')

In [7]:
from IPython.display import display, HTML
display(expected_result)

,Borough,Neighborhood,Postcode,Latitude,Longitude
0,North York,Henry Farm,M2J,43.778517,-79.346556
1,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790
2,Etobicoke,Markland Wood,M9C,43.643515,-79.577201
3,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242
4,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",M1P,43.757410,-79.273304
5,Etobicoke,Kingsview Village,M9R,43.688905,-79.554724
6,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
7,Etobicoke,"Humber Bay, King's Mill Park, Mimico NE, Old M...",M8Y,43.636258,-79.498509
8,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636
9,North York,Glencairn,M6B,43.709577,-79.445073


<h2>Part III</h2>

Now let's populate the map and cluster our data'

In [8]:
latitudeofToronto = 43.6531
longitudeofToronto = -79.3832
TorontoFolium = folium.Map(location = [latitudeofToronto, longitudeofToronto], zoom_start = 10.6)

for lat, lng, borough, neighborhood in zip(expected_result['Latitude'], expected_result['Longitude'], expected_result['Borough'], expected_result['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7).add_to(TorontoFolium)  
    

TorontoFolium

In [9]:
CLIENT_ID = 'MYQ3QLZXFN1OXIR5SFF4FIY1FWDTBWVUXYCVF15OS3RNGRAD' # your Foursquare ID
CLIENT_SECRET = 'UQ1HICEZTBTBTDVR3ZFFHFFFMKICIM4DEXENDCB3VKNAVHXR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [10]:
selected_toronto_boroughs = expected_result['Borough'].str.contains('Toronto')
withTorontoinName = expected_result.loc[selected_toronto_boroughs]
withTorontoinName.head(10)

,Borough,Neighborhood,Postcode,Latitude,Longitude
1,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790
8,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636
11,West Toronto,High Park,M6P,43.661608,-79.464763
15,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",M5K,43.647177,-79.381576
17,Central Toronto,Moore Park,M4T,43.689574,-79.383160
25,Central Toronto,"Deer Park, Rathnelly, South Hill",M4V,43.686412,-79.400049
26,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
28,Downtown Toronto,Ryerson,M5B,43.657162,-79.378937
30,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
36,Downtown Toronto,"Adelaide, King",M5H,43.650571,-79.384568


In [11]:
withTorontoinName.shape

(29, 5)

In [12]:
#update the map with the selected data: we look at the boroughs the names of which contain "Toronto"

TorMap = folium.Map(location=[latitudeofToronto, longitudeofToronto], zoom_start=12)
for lat, lng, label in zip(withTorontoinName['Latitude'], withTorontoinName['Longitude'], withTorontoinName['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7).add_to(TorMap)  
TorMap

In [13]:
withTorontoinName.tail()

,Borough,Neighborhood,Postcode,Latitude,Longitude
72,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",M5T,43.653206,-79.400049
74,East Toronto,The Beaches,M4E,43.676357,-79.293031
75,Downtown Toronto,"Cabbagetown, St. James Town",M4X,43.667967,-79.367675
77,East Toronto,India Bazaar,M4L,43.668999,-79.315572
83,Central Toronto,Forest Hill North,M5P,43.696948,-79.411307


Let's choose a random location and use Foursquare to get the data about the most popular venues in the area.

In [14]:
withTorontoinName.reset_index(level=None, drop=False, inplace=True, col_level=0, col_fill='')
N_latitude = withTorontoinName.loc[0, 'Latitude'] 
N_longitude = withTorontoinName.loc[0, 'Longitude'] 
N = withTorontoinName.loc[0, 'Neighborhood'] 
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitudeofToronto, longitudeofToronto, VERSION, radius, LIMIT)

In [15]:
venues = requests.get(url).json()
venues

{'meta': {'code': 200, 'requestId': '5bc88a0f1ed2194288858093'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5227bb01498e17bf485e6202-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
          'suffix': '.png'},
         'id': '4f2a25ac4b909258e854f55f',
         'name': 'Neighborhood',
         'pluralName': 'Neighborhoods',
         'primary': True,
         'shortName': 'Neighborhood'}],
       'id': '5227bb01498e17bf485e6202',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 169,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng': -79.38529600606677}],
        'lat': 43.6532

Now let's work with the json file

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
from pandas.io.json import json_normalize

venues = venues["response"]['groups'][0]['items'] 
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
3,Old City Hall,Monument / Landmark,43.652009,-79.381744
4,Indigo,Bookstore,43.653515,-79.380696


Now let's get the data for all neighborhoods from our dataset

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
withTorontoinName_venues = getNearbyVenues(names=withTorontoinName['Neighborhood'],
                                   latitudes=withTorontoinName['Latitude'],
                                   longitudes=withTorontoinName['Longitude']
                                  )

Lawrence Park
Harbourfront, Regent Park
High Park
Design Exchange, Toronto Dominion Centre
Moore Park
Deer Park, Rathnelly, South Hill
Church and Wellesley
Ryerson
St. James Town
Adelaide, King
Commerce Court
Runnymede, Swansea
Riverdale
Dovercourt Village
Berczy Park
The Annex, Yorkville
Toronto Islands, Union Station
University of Toronto
Little Portugal, Trinity
CN Tower, King and Spadina, Railway Lands, South Niagara
Parkdale, Roncesvalles
Rosedale
First Canadian Place, Underground city
Exhibition Place, Parkdale Village
Chinatown, Grange Park, Kensington Market
The Beaches
Cabbagetown, St. James Town
India Bazaar
Forest Hill North


In [21]:
withTorontoinName_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"CN Tower, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",46,46,46,46,46,46
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Church and Wellesley,88,88,88,88,88,88
Commerce Court,100,100,100,100,100,100
"Deer Park, Rathnelly, South Hill",15,15,15,15,15,15
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100


In [22]:
print('There are {} uniques categories.'.format(len(withTorontoinName_venues['Venue Category'].unique())))

There are 222 uniques categories.


In [23]:
# one hot encoding
tor_onehot = pd.get_dummies(withTorontoinName_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = withTorontoinName_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [24]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped.head(7)

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,"Adelaide, King",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.0,0.0,0.00,0.010000,0.01,0.000000,0.02,0.0,0.000000,0.020000,0.000000,0.03,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.010000,0.000000,0.01,0.020000,0.0,0.000000,0.00,0.020000,0.010000,0.0,0.000000,0.060000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.020000,0.010000,0.070000,0.0,0.0,0.0,0.000000,0.0,0.020000,0.0,0.030000,0.000000,0.0,0.0,0.000000,0.020000,0.01,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00,0.00,0.0,0.01,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.01,0.00,0.000000,0.000000,0.0,0.0,0.01,0.00,0.0,0.020000,0.000000,0.000000,0.01,0.0,0.000000,0.01,0.00,0.010000,0.000000,0.030000,0.01,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.

Let's return Top 5 venues per neighborhood based on the example in the course.


num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

And then select Top 10 for each neighborhood and put the data into a dataframe

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
import numpy as np 

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Gym,Cosmetics Shop,Bar,Restaurant,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Beer Bar,Bakery,Steakhouse,Restaurant,Café,Farmers Market
2,"CN Tower, King and Spadina, Railway Lands, Sou...",Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Boutique,Airport Food Court,Harbor / Marina
3,"Cabbagetown, St. James Town",Restaurant,Coffee Shop,Italian Restaurant,Pizza Place,Indian Restaurant,Pub,Park,Bakery,Market,Café
4,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Vietnamese Restaurant,Bakery,Mexican Restaurant,Dumpling Restaurant,Coffee Shop,Caribbean Restaurant
5,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Fast Food Restaurant,Gastropub,Men's Store,Pub
6,Commerce Court,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Seafood Restaurant,Gym,Steakhouse,Gastropub
7,"Deer Park, Rathnelly, South Hill",Coffee Shop,Pub,Pizza Place,Restaurant,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint,American Restaurant
8,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Hotel,Café,American Restaurant,Deli / Bodega,Sports Bar,Gastropub,Restaurant,Gym,Italian Restaurant
9,Dovercourt Village,Supermarket,Bakery,Gas Station,Café,Bank,Middle Eastern Restaurant,Bar,Discount Store,Gym / Fitness Center,Pharmacy


## Finally let's cluster the neighborhoods!

In [29]:

tor_grouped 

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,"Adelaide, King",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.010000,0.01,0.000000,0.020000,0.000000,0.000000,0.020000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.00,0.020000,0.010000,0.00,0.000000,0.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.020000,0.010000,0.070000,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.020000,0.01,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.00,0.000000,0.000000,0.000000,0.00000,0.010000,0.00,0.000000,0.0

In [30]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [42]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

tor_data = withTorontoinName.drop(16)
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:25] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 1, 0, 3, 0, 0, 2,
       0, 0, 0])

In [43]:
tor_merged = withTorontoinName

# add clustering labels
tor_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() # check the last columns!

C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,index,Borough,Neighborhood,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790,0,Park,Swim School,Dim Sum Restaurant,Bus Line,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
1,8,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Café,Restaurant,Pub,Mexican Restaurant,Breakfast Spot,Theater,Health Food Store
2,11,West Toronto,High Park,M6P,43.661608,-79.464763,0,Café,Bar,Mexican Restaurant,Fast Food Restaurant,Antique Shop,Bookstore,Sandwich Place,Diner,Flea Market,Italian Restaurant
3,15,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",M5K,43.647177,-79.381576,0,Coffee Shop,Hotel,Café,American Restaurant,Deli / Bodega,Sports Bar,Gastropub,Restaurant,Gym,Italian Restaurant
4,17,Central Toronto,Moore Park,M4T,43.689574,-79.383160,0,Gym,Trail,Playground,Intersection,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [45]:
# create map
map_clusters = folium.Map(location=[latitudeofToronto, longitudeofToronto], zoom_start=11)


x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We get the split into clusters. Far from perfect. But this is what I got. Looking for suggestions and improvements. 